# Langchain + FAISS + Ollama
This workbook demonstrate RAG with FAISS & ollama (offline)

**Notes:**
1. Install ollama: https://ollama.com/download
2. Download ollama embed: `ollama pull nomic-embed-text`
3. Download the model (eg. `ollama pull mistral-openorca`)
4. `ollama serve`

In [1]:
import os
import glob

import gradio as gr

In [2]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

from langchain.vectorstores import FAISS

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

## Directory setup

In [3]:
folders = glob.glob("knowledge-base/*")

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [4]:
len(documents)

31

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1300, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [6]:
len(chunks)

91

In [7]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: contracts, company, products, employees


## Convert to vector database
use `nomic-embed-text' from ollama library: https://ollama.com/library/nomic-embed-text.

In [8]:
from langchain_ollama import OllamaEmbeddings, ChatOllama

db_name = "knowledge_base_1"
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [9]:
# Create our FAISS vectorstore!

vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d

print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

There are 91 vectors with 768 dimensions in the vector store


## Chat setup


In [21]:
# create a new Chat with Ollama
from langchain_core.callbacks import StdOutCallbackHandler

MODEL = "mistral-openorca"
# MODEL = "llama3.2" # fail to answer

llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever(search_kwargs={"k": 13})

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, 
                                                           retriever=retriever, 
                                                           memory=memory, 
                                                           callbacks=[StdOutCallbackHandler()])

In [22]:
# run a quick test - should return a list of documents = 4
question = "Insurellm"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [23]:
docs[0]

Document(id='35e35da8-0bf9-4d18-9880-c5f4d5c77e7a', metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}, page_content="# About Insurellm\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. It's first product was Markellm, the marketplace connecting consumers with insurance providers.\nIt rapidly expanded, adding new products and clients, reaching 200 emmployees by 2024 with 12 offices across the US.")

In [24]:
# Chat test
query = "tell me about Insurellm"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
# Overview of Insurellm

Insurellm is an innovative insurance tech firm with 200 employees across the US.
Insurellm offers 4 insurance software products:
- Carllm, a portal for auto insurance companies
- Homellm, a portal for home insurance companies
- Rellm, an enterprise platform for the reinsurance sector
- Marketllm, a marketplace for connecting consumers with insurance providers
  
Insurellm has more than 300 clients worldwide.

# About Insurellm

Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. It's first product was Markellm, the marketplace connec

In [15]:
# Clearing memory before new chat
memory.clear()

In [16]:
def chat_gradio(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [17]:
# Ui launch
view = gr.ChatInterface(chat_gradio, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
